In [6]:
import os
import torch
import numpy as np
import csv

def model(t_u, w, b):
    return w * t_u + b

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

torch.set_printoptions(sci_mode=False)

t_c = torch.tensor([0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4])

# w = torch.ones((), requires_grad=True)
# b = torch.zeros((), requires_grad=True)

# t_p = model(t_u, w, b)
# loss = loss_fn(t_p, t_c)
# loss.backward()
# w.grad, b.grad

# delta = 0.1
# learning_rate = 1e-2

# loss_rate_of_change_w = \
#     (loss_fn(model(t_u, w + delta, b), t_c) -
#      loss_fn(model(t_u, w - delta, b), t_c)) / (2.0 * delta)
# w = w - learning_rate * loss_rate_of_change_w

# loss_rate_of_change_b = \
#     (loss_fn(model(t_u, w, b + delta), t_c) -
#      loss_fn(model(t_u, w, b - delta), t_c)) / (2.0 * delta)
# b = b - learning_rate * loss_rate_of_change_b



In [8]:
import torch

# def model(t_u, w, b):
#     return w * t_u + b

# def loss_fn(t_p, t_c):
#     squared_diffs = (t_p - t_c)**2
#     return squared_diffs.mean()

t_un =  t_u * 0.1
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:
            params.grad.zero_()
        
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        loss.backward()

        with torch.no_grad():
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
    return params

# torch.set_printoptions(sci_mode=False)

# t_c = torch.tensor([0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0])
# t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4])

learning_rate = 1e-2
params = torch.tensor([1.0, 0.0], requires_grad=True)
params = training_loop(n_epochs = 5000, learning_rate = learning_rate, params = params, t_u = t_un, t_c = t_c)
print(params)


Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647
tensor([  5.3671, -17.3012], requires_grad=True)


In [14]:
import torch.optim as optim

def training_loop1(n_epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print('Epoch {}, Loss: {}'.format(epoch, loss))
    
    return params

params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr = learning_rate)
params = training_loop1(n_epochs=5000, optimizer=optimizer, params=params, t_u=t_un, t_c=t_c)
print(params)
        

Epoch 500, Loss: 24.946460723876953
Epoch 1000, Loss: 15.752771377563477
Epoch 1500, Loss: 9.455153465270996
Epoch 2000, Loss: 5.767541885375977
Epoch 2500, Loss: 3.932838201522827
Epoch 3000, Loss: 3.196812391281128
Epoch 3500, Loss: 2.9771506786346436
Epoch 4000, Loss: 2.933181047439575
Epoch 4500, Loss: 2.9279677867889404
Epoch 5000, Loss: 2.9276540279388428
tensor([  5.3660, -17.2952], requires_grad=True)


In [21]:
def training_loop2(n_epochs, optimizer, params, train_t_u, val_t_u, trina_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss =loss_fn(train_t_p, train_t_c)
        
        val_t_p = model(val_t_u, *params)
        val_loss = loss_fn(val_t_p, val_t_c)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print('Epoch {}, Training loss: {}, Validation loss: {}'.format(epoch, train_loss, val_loss))

    return params

n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr = learning_rate)
params = training_loop2(n_epochs=5000, optimizer=optimizer, params=params, train_t_u= train_t_un, trina_t_c= train_t_c, val_t_u=val_t_un, val_t_c = val_t_c)
print(params)

Epoch 500, Training loss: 7.161751747131348, Validation loss: 2.25825834274292
Epoch 1000, Training loss: 3.470465898513794, Validation loss: 2.71073579788208
Epoch 1500, Training loss: 3.0020039081573486, Validation loss: 2.90449595451355
Epoch 2000, Training loss: 2.942551612854004, Validation loss: 2.977656602859497
Epoch 2500, Training loss: 2.935004472732544, Validation loss: 3.0042498111724854
Epoch 3000, Training loss: 2.9340462684631348, Validation loss: 3.0137853622436523
Epoch 3500, Training loss: 2.933924674987793, Validation loss: 3.0171902179718018
Epoch 4000, Training loss: 2.933910369873047, Validation loss: 3.0184054374694824
Epoch 4500, Training loss: 2.933908224105835, Validation loss: 3.018829822540283
Epoch 5000, Training loss: 2.933908700942993, Validation loss: 3.0189974308013916
tensor([  5.4164, -17.4319], requires_grad=True)
